# KNN with Solar Power Ramp Events

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

## Importing the dataset

In [2]:
os.getcwd()
os.chdir('../datasets')
# dataset = pd.read_csv('Social_Network_Ads.csv')
dataset = pd.read_csv('rampdata66.csv')
# X = dataset.iloc[:, [2, 3]].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 66].values
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V58,V59,V60,V61,V62,V63,V64,V65,V66,Class
0,0.001610,0.001852,0.445164,0.710982,0.482693,0.346400,0.786719,0.296912,0.212884,0.414958,...,4,2,2,2,2,2,1,2,2,4
1,0.127630,0.085961,0.548310,0.907729,0.999993,0.607910,0.184483,0.370349,0.051022,0.696017,...,2,2,2,2,2,4,4,2,2,2
2,0.034725,0.109288,0.510308,0.922363,0.999993,0.598586,0.159467,0.365388,0.054600,0.720091,...,2,2,2,2,2,2,2,2,2,2
3,0.033372,0.276992,0.473963,0.927030,0.999993,0.604930,0.140143,0.369037,0.092530,0.730662,...,4,4,4,2,2,2,2,2,4,4
4,0.171606,0.153857,0.456074,0.928770,0.999993,0.595534,0.121081,0.371288,0.086637,0.736614,...,2,4,4,2,2,2,2,4,4,4


## Encoding categorical data

In [3]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# labelencoder_X = LabelEncoder()
# X[ : , 1] = labelencoder_X.fit_transform(X[ : , 1])
# X

## Splitting the dataset into the Training set and Test set

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Cross-validation for KNN: 
2folds for each of 100 candidates, totalling 200 fits, (it takes time about 5mins)

https://scikit-learn.org/stable/modules/model_evaluation.html

classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

In [6]:
# from sklearn import metrics   ##Additional scklearn functions
# from sklearn.model_selection import cross_validate
# from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.neighbors import KNeighborsClassifier
# # KNN_model = KNeighborsClassifier()

# test_params = {'n_neighbors':[1, 2, 3, 4, 5, 6], p:[0.05, 0.1, 0.2, 0.3, 0.5]}

# grid = GridSearchCV(estimator = KNN_model, param_grid = test_params, n_jobs=4, 
#                     scoring='f1_weighted',cv=2, verbose=3)

# grid.fit(X_train,y_train)
# print(grid.best_params_)

### Fitting KNN  
Evaluate metric(s) by cross-validation and also record fit/score times.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

https://scikit-learn.org/stable/modules/cross_validation.html

In [17]:
from sklearn import metrics   ##Additional scklearn functions
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
params = {'n_neighbors': 5 }
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
cv_results = cross_validate(classifier, X, y, cv=2)
sorted(cv_results.keys())                          

['fit_time', 'score_time', 'test_score', 'train_score']

In [18]:
cv_results

C:\Users\Mhdella\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.01097322, 0.00698233]),
 'score_time': array([0.22245145, 0.19968843]),
 'test_score': array([0.86057441, 0.86774699]),
 'train_score': array([0.9012023 , 0.87885117])}

In [19]:
sum(y_train.shape+y_test.shape)

3828

## Single metric evaluation using cross_validate

In [20]:
sorted(cv_results.keys())                         
['fit_time', 'score_time', 'test_score']
cv_results['test_score']    

array([0.86057441, 0.86774699])

####  Multiple metric evaluation using cross_validate 

In [21]:
scores = cross_validate(classifier, X, y, cv=2, scoring=('f1_weighted'), return_train_score=True)

In [22]:
# print(scores['train_f1_weighted'])   
# print(scores['f1_weighted'])   
scores
# train_f1_weighted

{'fit_time': array([0.01196933, 0.00697351]),
 'score_time': array([0.22521448, 0.18650699]),
 'test_score': array([0.85881854, 0.86414623]),
 'train_score': array([0.89987304, 0.87732008])}

## Making the Confusion Matrix

In [23]:
>>> from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(classifier, X, y, cv=2)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, y)
cm

array([[  56,   38,    0,    5],
       [  74, 1085,    1,  205],
       [   0,    1,    5,    4],
       [   1,  166,   25, 2162]], dtype=int64)

In [24]:
print('True Hi_Ramps=',cm[0,0]+cm[2,2])
print('False Hi_Ramps=',cm[1,0]+cm[2,0]+cm[3,0]+cm[0,2]+cm[1,2]+cm[3,2])

True Hi_Ramps= 61
False Hi_Ramps= 101


In [25]:
from sklearn.metrics import f1_score
f1_score(y, y_pred, average=None)

array([0.48695652, 0.8173258 , 0.24390244, 0.9141649 ])

## Analysis on the outcomes y, (classes of solar ramp events)

In [26]:
y.shape
len(y)
print('C1=', len(y[y==1]),'C2=', len(y[y==2]),
      'C3=', len(y[y==3]),'C4=', len(y[y==4]))

C1= 131 C2= 1290 C3= 31 C4= 2376
